# **`BLiMP Checkout`**

**BLiMP TEST ORDER:**
1. Todo

**Default Settings:**
\
`wired_seq_num = 0x01`
\
`itr=1`
\
`send_data_packet_to_wd_before_sniffing = True`
\
`specific_cmd_name_override = None`

**To sniff Serial from SBC without sending a command:**
1. `send_data_packet_to_wd_before_sniffing = False`

**To enter SERVICE mode:**
1. `itr = 1`

**To enter MISSION mode:**
1. `itr = 2`

**System Soft Reset:**  *(All systems except the WatchDog powered off)*
1. Enter **`SERVICE`** mode twice

**To Reset Specific Systems:**
1. specific_cmd_name_override = 'power-off-[system]`   
    *System can be one of the following:*
    - `herc`
    - `radio`
    - `motors`
    - `cameras`

**No telem from Herc after 200 sec**
1. `specific_cmd_name_override = 'power-off-herc'`
2. `specific_cmd_name_override = 'power-off-radio'`
3. `specific_cmd_name_override = 'power-on-radio'`
4. `specific_cmd_name_override = 'power-on-herc'`

**Misc Commands:**
\
See ***Pre-prepared commands list***.

In [1]:
## SETUP:

from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

wired_seq_num = 0x01
wireless_seq_num = wired_seq_num
blimp_cmds = ['setup', 'power-on', 'SetChargerEn', 'SetChargerPowerConnection', 'SetBatteryConnection', 'SetBatteryControlEnable', 'SetBatteryLatch']
cmd = blimp_cmds[1]
blimp_inputs = [False, True, 'OFF', 'ON', 'FORCE_HIGH', 'FORCE_LOW']
param = blimp_inputs[0]

send_data_packet_to_wd_before_sniffing = False
specific_cmd_name_override = cmd

  

## CE Test One - All Internal Rover Systems Pre-Deployment Enabled
# specific_cmd_name_override = 'setup' # ~8         32
# specific_cmd_name_override = 'power-on' # ~14.5   38

# specific_cmd_name_override = 'power-off-herc' #    
# specific_cmd_name_override = 'power-off-radio' # 
# specific_cmd_name_override = 'power-on-herc' #    44
# specific_cmd_name_override = 'power-on-radio' #   45

# specific_cmd_name_override = 'reset-radio'
# specific_cmd_name_override = 'reset-herc' #         59 -> 63
## FAILED

# specific_cmd_name_override = 'SetBatteryConnection'
# param = False

## CE Test One - All External Rover Systems Post-Deployment Enabled
## CE Test Two - All Rover Systems Enabled
## CE Test Three - 

# specific_cmd_name_override = 'set-CE'

# REGE_set = False
# BE_set = False
# CE_set = 'OFF' # 'OFF' 'ON' 'FORCE_HIGH'
# BCE_set = 'OFF' # 'OFF' 'ON' 'FORCE_HIGH'
# BL_set = 'OFF' # 'OFF' 'ON' 'PULSE_HIGH'


print(specific_cmd_name_override,type(specific_cmd_name_override),'  :  ',param,type(param))

power-on <class 'str'>   :   False <class 'bool'>


In [2]:
## SETTINGS:

serial_device = '/dev/ttyUSB1'
ip="192.168.1.2"
port=8080

In [3]:
settings['SAVE_FILE_PREFIX'] = 'iris__csTestOne__10_28_21' # this is the prefix on all log files. make it something unique.
load_cache()

# IF ERROR - update last number in FILE_PREFIX

In [4]:
## SET TO TRUE TO VIEW A FULL-LIST OF ALL COMMANDS AND TELEMETRY:
module_to_lookup = 'WatchDogInterface'

if show_commands := False:
    standards.print_overview()
if show_module_standards := False:
        def module(x): return cprint(f"\n\t{x}", 'magenta', 'on_grey', attrs=['bold'])
        def header(x): return cprint(f"\n\t\t{x}", 'grey', 'on_white')
        def command(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'green')
        def telemetry(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'red')
        def event(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'blue')

        def p_arg(a): return cprint(f"\n\t\t\t\t\tAvailable values for `{a.name}`:", 'cyan')

        def p_enum(x): return cprint(f"\n\t\t\t\t\t\t'{x.name}' or {x.value} or {hex(x.value)}", 'magenta')

        print("Data Standards Overview: [")
        m = standards.modules[module_to_lookup]
        module(m)
        header('Commands:')
        for i, c in enumerate(m.commands.vals):
            command(i, c)
            for arg in c.args:
                if len(arg.enum) > 0:
                    p_arg(arg)
                    for e in arg.enum:
                        p_enum(e)
        header('Telemetry:')
        for i, t in enumerate(m.telemetry.vals):
            telemetry(i, t)
        header('Events:')
        for i, ev in enumerate(m.events.vals):
            event(i, ev)
        print('\n]')

## **Testing Procedure:** picking pre-prepared commands

**Running the Code:**

- On the first run **OR** after resetting / power-cycling the board, **pick a step in the next cell** then press the `red square`, then `green circle`, then `double arrows` to run all the code.
- After changing to the next step (in the next cell), **click here** (yes, this text cell that you're currently reading) and then **press only** the `red square` then the `play button with the down arrow` to send the new command without resetting the sequence numbers.
- You can read wired telemetry streams at the bottom of this notebook and check for the presence of wireless telemetry in wireshark using the filter: `udp && udp.port==8080 && not icmp` . 

**Steps:**

0. Run `setup`, wait for watchdog heartbeats to verify that the mode is now `RS_SERVICE`.
1. Run `power-on`, verify Hercules has been turned on by the presense of `ICP` telemetry packets in the wired output.
2. Run `wifi-mode` to tell Hercules to downlink telemetry wirelessly instead of over the wire. Verify that `ICP` telemetry packets are now showing up in the wireless output. Note: there should no longer be a wired output showing. Also note: wired and wireless commands will still work until the wired connection has been physically disconnected.
3. Run `deploy` to tell Hercules to tell the Watchdog to release its deployment interlock and then have Hercules release its own interlock. Verify that both interlocks are released and that the deployment switch is now closed by removing the deployment power leads from the deployment power supply and verifying that the resistance between them is on the order of single digit Ohms instead of Megaohms.

Once `deploy` has been successfully issued, applying deployment power to the rover will cause its deployment. **Note:** to properly simulate mission conditions, the interlocks should be released and deployment switch closed **before** applying deployment power.

**Troubleshooting:**

**Note:** If you have issues running the `deploy` command, you can instead run the `deploy-wd-only` and the `deploy-herc-only` commands to directly tell the Watchdog and Hercules to release their interlocks separately.

**Note:** If you're having trouble getting wireless comms working, make sure this computer's IP matches the `spacecraft_ip` used by Hercules (usually `192.168.1.120`) and that the IP you're sending to (configured at the top) matches the `rover_ip` used by the Hercules.

In [5]:
## Pick the step you want to run:

steps = ['heater-test', 'setup', 'power-on', 'wifi-mode', 'deploy', 'misc-test']
if specific_cmd_name_override is not None:
    step = specific_cmd_name_override
else:
    step = steps[itr]
step

'power-on'

In [6]:
## Pre-prepared commands list:

'''ACCIDENTAL PASTE'''

source = DataSource.GENERATED

prepared_commands = {
    'setup': ( # Tell the Watchdog to switch into service mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SwitchToServiceMode',
        dict(confirm='CONFIRM_SERVICE'),
        DataPathway.WIRED
    ),
    'power-on': ( # Tell the Watchdog to switch into mission mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_PrepareForDeployment',
        dict(confirm='CONFIRM_PREP'),
        DataPathway.WIRED
    ),
    'wifi-mode': ( # Command herc to send telem over wifi
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WF_121'),
        DataPathway.WIRELESS
    ),
    'wired-mode': ( # Command herc to send telem over uart
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WATCHDOG'),
        DataPathway.WIRELESS
    ),
    'deploy': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRED
    ),
    'reset-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_HERCULES'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'reset-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_RADIO'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RADIO_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RADIO_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'reset-fpga': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_FPGA'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '3v3-on': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_3_3_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '3v3-off': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_3_3_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'SetChargerEn': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SetChargerEn',
        dict(charge_en=param),
        DataPathway.WIRED
    ),
    'SetChargerPowerConnection': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SetChargerPowerConnection',
        dict(v_lander_reg_en=param),
        DataPathway.WIRED
    ),
    'SetBatteryConnection': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SetBatteryConnection',
        dict(batt_en=param),
        DataPathway.WIRED
    ),
    'SetBatteryControlEnable': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SetBatteryControlEnable',
        dict(batt_ctrl_en=param),
        DataPathway.WIRED
    ),
    'SetBatteryLatch': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SetBatteryLatch',
        dict(latch_batt=param),
        DataPathway.WIRED
    ),
    'misc-test': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='BATTERY_START_CHARGE'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    )
}

pathway, magic, command_name, kwargs, telem_pathway = prepared_commands[step]

In [7]:
## Build Command:

'''ACCIDENTAL PASTE'''

if pathway == DataPathway.WIRED:
    seq_num = wired_seq_num
elif pathway == DataPathway.WIRELESS:
    seq_num = wireless_seq_num

command_payload_type = {
    Magic.WATCHDOG_COMMAND: WatchdogCommandPayload,
    Magic.COMMAND: CommandPayload
}[magic]

module, command = standards.global_command_lookup(command_name)

payloads = PayloadCollection(
    CommandPayload=[
        command_payload_type(
            pathway=pathway,
            source=source,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
)
packet = IrisCommonPacket(
    seq_num = seq_num, 
    payloads = payloads,
    pathway = pathway,
    source = source
).encode()
print(hexstr(packet)) # 02 07 00 d6 ee ff 00 c0 ec 10 77

01 07 00 d9 ee ff 00 c0 01 10 60


In [8]:
## Send Command:
connect_serial(device = serial_device)
from IrisBackendv3.data_standards import DataStandards
if send_data_packet_to_wd_before_sniffing:
    if pathway == DataPathway.WIRED:
        send_data_wd_serial(packet)
    elif pathway == DataPathway.WIRELESS:
        send_wifi(packet, ip=ip, port=port)

Connection Success!


In [9]:
## Stream Telemetry:
if True or telem_pathway == DataPathway.WIRED:
    stream_data_ip_udp_serial()
elif telem_pathway == DataPathway.WIRELESS:
    pass # Check for wireless telemetry in Wireshark. For parsing help, run: `pyenv exec python parse_pcap.py --help`

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1

2021-10-28 17:00:06 Houston IrisBackendv3.codec.logging[7020] WARNING Invalid packet detected. Does not conform to any supported specs: b'\xff\xff\n\x07\xa0\xd8\x00\x00\x00\x00\x00\x00\xf4\x01\xfd\x0c<\x004!\x10\x00\x01\x00\x00'


RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: OFF, Ctrl: ON] 	298.1